In [6]:
import numpy as np

In [7]:
#Including numba to make the program more effective
import numba as nb
from numba import jit
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [8]:
#Particle - particle collision:
@jit(nopython=True)
def particleCollisionInitialize(particles, collision_count):
    '''
    Parameters:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1]
                     [posx2, posy2, velx2, vely2, mass2, radius2]
                     ...
                     [posxN, posyN, velxN, velyN, massN, radiusN]]
        collision_count = 

    Output:
        array_of_min_collision_times = [[col.time particle1, particle1, particle it collided with, wall = 2, col. count particle1, col. count other particle]
                                        [col.time particle2, particle2, particle it collided with, wall = 2, col. count particle2, col. count other particle]
                                        
                                        ...
                                        [col.time particleN, particleN, particle it collided with, wall = 2, col. count particleN, col. count other particle]
                                        
                                        wall = 2 represent particle collision 

    Finds the minimum time til collision with other particles for each particle in the system.
    '''

    array_of_min_collision_times = np.zeros((len(particles),6)) #Prøva å få på samme form som med wall
    
    for i in range(len(particles)):

        array_next_dt = np.zeros((len(particles)-i-1,2)) 

        x_pos_i = float(particles[i,0])
        y_pos_i = float(particles[i,1])
        x_vel_i = float(particles[i,2])
        y_vel_i = float(particles[i,3])

        radius_i = float(particles[i,5])
        
        ink = -1 
        for j in range(len(particles)):
            if j != i and j > i:
                
                ink += 1 

                x_pos_j = float(particles[j,0])
                y_pos_j = float(particles[j,1])
                x_vel_j = float(particles[j,2])
                y_vel_j = float(particles[j,3])

                radius_j = float(particles[j,5])

                R_ij_2 = (radius_i + radius_j)**2


                deltaPos = np.array([x_pos_i - x_pos_j, y_pos_i - y_pos_j])
                deltaVel = np.array([x_vel_i - x_vel_j, y_vel_i - y_vel_j])


                deltaPos_deltaPos = np.dot(deltaPos,deltaPos)
                deltaPos_deltaVel = np.dot(deltaPos,deltaVel)
                deltaVel_deltaVel = np.dot(deltaVel,deltaVel)


                d = deltaPos_deltaVel**2 - (deltaVel_deltaVel)*(deltaPos_deltaPos - R_ij_2)

                
                if deltaPos_deltaVel < 0 and d > 0:
                    time_calc = -(deltaPos_deltaVel + np.sqrt(d))/deltaVel_deltaVel 
                    array_next_dt[ink] = np.array([time_calc, j])
                else:
                    array_next_dt[ink] = np.array([np.inf, j])
        

        if len(array_next_dt) != 0:
            min_time = np.amin(array_next_dt[:,0])
            index_other_particle = np.argmin(array_next_dt[:,0])
            other_particle =int(array_next_dt[index_other_particle, 1])
        else:
            min_time = np.inf
            other_particle = i

        array_of_min_collision_times[i] = [min_time, i, other_particle, 2, collision_count[i], collision_count[other_particle]]

    return array_of_min_collision_times

In [11]:
#@jit(nopython=True)
def calculate_next_collision_times(particles, particle1, particle2, collision_count, timestep, elasticity = 1): 
    '''
    Parameters:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1]
                     [posx2, posy2, velx2, vely2, mass2, radius2]
                     ...
                     [posxN, posyN, velxN, velyN, massN, radiusN]]
        particle1, particle2 = Particles involved in collision
        collision_count = The number of times each particle has collided
        timestep = Last calculated timestep

    Output:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1]
                        [posx2, posy2, velx2, vely2, mass2, radius2]
                        ...
                        [posxN, posyN, velxN, velyN, massN, radiusN]]
        new_collision_times_particles = All calculated collision times
        particle1, particle2 = Particles involved in collision

    Function that calculates new velocities and new collision times for the two particles involved in a
    particle-particle collision.
    '''

    m_1 = particles[particle1, 4]  #sto 5
    m_2 = particles[particle2, 4]

    radius = np.array([float(particles[particle1,5]),float(particles[particle2,5])])
    
    #Finding new velocities using (11)
    
    #Particle 1
    vel_old_1 = np.array([float(particles[particle1][2]), float(particles[particle1][3])])
    pos_old_1 = np.array([float(particles[particle1][0]), float(particles[particle1][1])])

    #Particle 2
    vel_old_2 = np.array([float(particles[particle2][2]), float(particles[particle2][3])])
    pos_old_2 = np.array([float(particles[particle2][0]), float(particles[particle2][1])])

    #New positions
    next_pos_1 = np.array([pos_old_1[0] + vel_old_1[0]*timestep, pos_old_1[1] + vel_old_1[1]*timestep])
    next_pos_2 = np.array([pos_old_2[0] + vel_old_2[0]*timestep, pos_old_2[1] + vel_old_2[1]*timestep])

    #dv and dx, equation (8) and (9)
    dv = np.array([vel_old_1[0] - vel_old_2[0], vel_old_1[1] - vel_old_2[1]])
    dx = np.array([next_pos_1[0] - next_pos_2[0], next_pos_1[1] - next_pos_2[1]])
    dvdx = np.dot(dv,dx) 

    #R_ij
    R_ij_2 = (radius[0] + radius[1])**2

    #Equation (11):
    v_next_1 = np.array([vel_old_1[0] - ((1+elasticity)*(m_2/(m_1+m_2))*dvdx/R_ij_2)*dx[0],
                         vel_old_1[1] - ((1+elasticity)*(m_2/(m_1+m_2))*dvdx/R_ij_2)*dx[1]])

    v_next_2 = np.array([vel_old_2[0] + ((1+elasticity)*(m_1/(m_1+m_2))*dvdx/R_ij_2)*dx[0],
                         vel_old_2[1] + ((1+elasticity)*(m_1/(m_1+m_2))*dvdx/R_ij_2)*dx[1]])


    #Updating particles matrix:
    particles[particle1, 2:4] = v_next_1 
    particles[particle2, 2:4] = v_next_2
    
    particles[particle1, 0:2] = next_pos_1
    particles[particle2, 0:2] = next_pos_2

    particles = updateParticles(particles, np.array([particle1, particle2]), timestep)
    new_collision_times_particles = particleCollisionInitialize(particles, collision_count)
    
    return particles, new_collision_times_particles, particle1, particle2

In [10]:
@jit(nopython=True)
def new_time_after_collision_wall(particles, particle1, collision_count):
    '''
    Parameters:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1]
                     [posx2, posy2, velx2, vely2, mass2, radius2]
                     ...
                     [posxN, posyN, velxN, velyN, massN, radiusN]]
        particle1 = Particle that collided with wall
        collision_count = The number of times each particle has collided

    Output:
        new_collision_times_particles = All calculated collision times

    After a particle has collided with a wall the positions will be updated, so we then have to calculate collision times again.
    '''

    posx = float(particles[particle1, 0])
    posy = float(particles[particle1, 1])
    velx = float(particles[particle1, 2])
    vely = float(particles[particle1, 3])
    r1 = particles[particle1, 5]

    #Finding new collision times
    array_of_new_collision_times = np.zeros((1,6))

    #Finding new collision times
    new_collision_times_particles = particleCollisionInitialize(particles, collision_count)

    return new_collision_times_particles

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d17ccd91-5f3d-4d77-a625-6b1eb51e3637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>